In [1]:
import mbuild as mb
from foyer import Forcefield

In [2]:
water = mb.load('files/tip3p.mol2')
water.name = 'SOL'

/Users/raymatsumoto/anaconda3/envs/mosdef36/lib/python3.6/site-packages/mdtraj/formats/mol2.py:203: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col=0, header=None, sep="\s+", engine='python')


In [3]:
c_ff = Forcefield('files/carbon.xml')
spce = Forcefield('files/spce.xml')

In [4]:
system = mb.recipes.GraphenePoreSolvent(pore_depth=2,side_dim=2,
                    n_sheets=3,pore_width=1.2,x_bulk=1.5,
                    solvent=water,n_solvent=300)
box = mb.Box(system.periodicity)

/Users/raymatsumoto/mbuild/mbuild/lattice.py:526: UserWarning: Periodicity of non-rectangular lattices are not valid with default boxes. Only rectangular lattices are valid at this time.
  warn('Periodicity of non-rectangular lattices are not valid with '
/Users/raymatsumoto/mbuild/mbuild/box.py:34: UserWarning: Provided `lengths` and `mins` and/or `maxs`. Only `lengths` is being used. You provided: lengths=[1.9648     2.875      1.91426255] mins=None maxs=None
  "lengths={} mins={} maxs={}".format(lengths, mins, maxs)
/Users/raymatsumoto/mbuild/mbuild/box.py:34: UserWarning: Provided `lengths` and `mins` and/or `maxs`. Only `lengths` is being used. You provided: lengths=[4.9648     2.875      1.91426255] mins=None maxs=None
  "lengths={} mins={} maxs={}".format(lengths, mins, maxs)


In [5]:
water = mb.Compound()
graphene = mb.Compound()
for child in system.children:
    if child.name == 'SOL':
        water.add(mb.clone(child))
    else:
        graphene.add(mb.clone(child))

water_pmd = spce.apply(water, residues='SOL')           
pore_pmd = c_ff.apply(graphene)

/Users/raymatsumoto/anaconda3/envs/mosdef36/lib/python3.6/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


In [6]:
system = water_pmd + pore_pmd
system.box[:3] = box.maxs * 10.0
system.save('init.gro', overwrite=True, combine='all')
system.save('init.top', overwrite=True, combine='all')

In [7]:
!gmx grompp -f files/nvt.mdp -c init.gro -p init.top -o nvt.tpr

                  :-) GROMACS - gmx grompp, VERSION 5.1.2 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University

In [ ]:
!gmx mdrun -v -deffnm nvt